In [198]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import scipy.stats
from scipy.stats import zscore
import glob
import re


In [199]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 5)

In [200]:
script_dir = os.getcwd()

In [201]:
bids_dir = os.path.join(os.getcwd(), '..', 'ds003745')
bids_dir = os.path.abspath(bids_dir)

In [202]:
age_srndna_path = os.path.join(bids_dir, '..', 'participants-srndna.tsv')
age_rf1_path = os.path.join(bids_dir, '..', 'participants-rf1.tsv')

In [203]:
age_srndna = pd.read_table(age_srndna_path)
age_rf1 = pd.read_table(age_rf1_path)
age_metrics_df = pd.concat([age_srndna, age_rf1])
age_df = age_df.rename(columns={"participant_id": "sub"})
age_df['sub'] = age_df['sub'].astype(str).str.replace('^sub-', '', regex=True)
age_df['sub'] = age_df['sub'].astype(int)  # Convert back to integer if needed
display(age_df)

,sub,age,sex,group
0,106,20,F,control
1,109,28,F,control
...,...,...,...,...
210,11432,35,F,control
211,11450,45,M,control


In [204]:
# Load paths for FD mean, tsnr, and missing runs

exc_dir = os.path.join(os.getcwd(), '..', 'derivatives', 'exclusions')
exc_dir = os.path.abspath(exc_dir)

metrics = os.path.join(exc_dir, 'fmri-metrics.tsv')
metrics_df = pd.read_csv(metrics, sep='\t')
#display(metrics_df)

missedtrials = os.path.join(exc_dir, 'missing_trials_coding.csv')
missedtrials_df = pd.read_csv(missedtrials)
missedtrials_df.rename(columns={"Subject": "sub", "Run": "run", "Exclusion": "Status"}, inplace=True)
display(missedtrials_df)

,sub,run,computer_non-face.txt,computer_non-faceclea.txt,event_computer_neutral.txt,event_computer_punish.txt,event_computer_reward.txt,event_friend_neutral.txt,event_friend_punish.txt,event_friend_reward.txt,event_stranger_neutral.txt,event_stranger_punish.txt,event_stranger_reward.txt,friend_face.txt,missed_trial.txt,stranger_face.txt,Missed_Trial_Count,Status
0,10317,1,1.0,18.0,2.0,8.0,8.0,2.0,8.0,8.0,2.0,8.0,8.0,18.0,0.0,18.0,0,Include
1,10317,2,1.0,18.0,2.0,8.0,8.0,2.0,8.0,8.0,2.0,8.0,8.0,18.0,0.0,18.0,0,Include
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488,159,1,1.0,0.0,1.0,15.0,16.0,3.0,6.0,7.0,3.0,13.0,8.0,1.0,0.0,1.0,0,Include
489,159,2,1.0,0.0,1.0,7.0,8.0,6.0,13.0,13.0,4.0,8.0,12.0,1.0,0.0,1.0,0,Include


In [205]:
import pandas as pd

# Function to detect outliers using 1.5 * IQR
def detect_outliers(series):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return (series < lower_bound) | (series > upper_bound)

# Identify 3-digit and 5-digit subjects
metrics_df["sub_length"] = metrics_df["sub"].astype(str).apply(len)

# Compute outliers for fd_mean across all subjects
fd_outliers = detect_outliers(metrics_df["fd_mean"])

# Compute outliers for tsnr separately for 3-digit and 5-digit IDs
metrics_df["tsnr_outlier"] = False  # Initialize column
for length in [3, 5]:  # Process 3-digit and 5-digit groups separately
    mask = metrics_df["sub_length"] == length
    metrics_df.loc[mask, "tsnr_outlier"] = detect_outliers(metrics_df.loc[mask, "tsnr"])

# Assign 'Include' or 'Exclude'
metrics_df["Status"] = ["Exclude" if fd or tsnr else "Include" for fd, tsnr in zip(fd_outliers, metrics_df["tsnr_outlier"])]

# Drop helper columns if needed
metrics_df.drop(columns=["sub_length", "tsnr_outlier"], inplace=True)

display(metrics_df)



,sub,run,tsnr,fd_mean,Status
0,104,1,76.060087,0.081698,Include
1,104,2,79.484593,0.086234,Include
...,...,...,...,...,...
463,11376,1,37.676049,0.150879,Include
464,11376,2,37.286160,0.147187,Include


In [206]:

# Now that we have our list of inputs, we can pull out the proper tsnr and fdmean for each subject
# Metric values should be derived from the same L1 run as the L3 input, if L2, use the average of both runs
# We are also going to do the math in this cell for averaging the variables

# Pivot the DataFrame to reshape it
parsed_metrics_df = metrics_df.pivot(index='sub', columns='run')

# Flatten the MultiIndex columns
parsed_metrics_df.columns = [f'{var}_{run}' for var, run in parsed_metrics_df.columns]

# Create the averages and the columns for them
# parsed_metrics_df['avg_tsnr'] = parsed_metrics_df[['vsmean_stan_1', 'vsmean_stan_2']].mean(axis=1)
parsed_metrics_df['avg_fdmean'] = parsed_metrics_df[['fd_mean_1', 'fd_mean_2']].mean(axis=1)

# Print the reshaped DataFrame
display(parsed_metrics_df)

,tsnr_1,tsnr_2,fd_mean_1,fd_mean_2,Status_1,Status_2,avg_fdmean
sub,,,,,,,
104,76.060087,79.484593,0.081698,0.086234,Include,Include,0.083966
105,84.525386,101.482166,0.089282,0.085409,Include,Include,0.087345
...,...,...,...,...,...,...,...
11387,36.536574,36.972787,0.230438,0.211929,Include,Include,0.221184
11410,17.799139,16.902037,0.304367,0.411635,Include,Include,0.358001


In [207]:
# Select relevant columns
metrics_status = metrics_df[["sub", "run", "Status"]].rename(columns={"Status": "Status_metrics"})
missedtrials_status = missedtrials_df[["sub", "run", "Status"]].rename(columns={"Status": "Status_missedtrials"})

# Merge on 'sub' and 'run'
merged_status_df = pd.merge(metrics_status, missedtrials_status, on=["sub", "run"], how="outer")

# Determine final exclusion status
merged_status_df["status"] = merged_status_df.apply(
    lambda row: "Exclude" if row["Status_metrics"] == "Exclude" or row["Status_missedtrials"] == "Exclude" else "Include",
    axis=1
)

# Select relevant columns for output
exclusions = merged_status_df[["sub", "run", "status"]]

# Display the final DataFrame
#display(exclusions)

# Filter rows where status is 'Exclude'
excluded_runs = exclusions[exclusions["status"] == "Exclude"]

# Display the new DataFrame
display(excluded_runs.head(40))

,sub,run,status
11,109,2,Exclude
15,111,1,Exclude
...,...,...,...
459,11374,2,Exclude
460,11374,1,Exclude


In [208]:
fsl_inputs = os.path.join(script_dir, 'L3inputs.csv') # This df stores the relevant FSL inputs
fsl_df = pd.read_csv(fsl_inputs)
print(fsl_df)

                                                                                                                                                                           path
0    /gpfs/scratch/tug87422/smithlab-shared/sharedreward-aging/derivatives/fsl/sub-10317/L2_task-sharedreward_model-1_type-ppi_seed-VS_sm-4.gfeat/cope4.feat/stats/cope1.nii.gz
1    /gpfs/scratch/tug87422/smithlab-shared/sharedreward-aging/derivatives/fsl/sub-10369/L2_task-sharedreward_model-1_type-ppi_seed-VS_sm-4.gfeat/cope4.feat/stats/cope1.nii.gz
..                                                                                                                                                                          ...
230    /gpfs/scratch/tug87422/smithlab-shared/sharedreward-aging/derivatives/fsl/sub-158/L2_task-sharedreward_model-1_type-ppi_seed-VS_sm-4.gfeat/cope4.feat/stats/cope1.nii.gz
231    /gpfs/scratch/tug87422/smithlab-shared/sharedreward-aging/derivatives/fsl/sub-159/L2_task-sharedreward_model-1_ty

In [209]:
# This cell is responsbile for adding the correct L3 input to the subject row
fsl_df['sub'] = fsl_df['path'].str.extract(r'sub-(\d+)').astype(int)
fsl_df['run'] = fsl_df['path'].str.extract(r'run-(\d+)').fillna('3').astype(int)
display(fsl_df)

,path,sub,run
0,/gpfs/scratch/tug87422/smithlab-shared/sharedreward-aging/derivatives/fsl/sub-10317/L2_task-sharedreward_model-1_type-ppi_seed-VS_sm-4.gfeat/cope4.feat/stats/cope1.nii.gz,10317,3
1,/gpfs/scratch/tug87422/smithlab-shared/sharedreward-aging/derivatives/fsl/sub-10369/L2_task-sharedreward_model-1_type-ppi_seed-VS_sm-4.gfeat/cope4.feat/stats/cope1.nii.gz,10369,3
...,...,...,...
230,/gpfs/scratch/tug87422/smithlab-shared/sharedreward-aging/derivatives/fsl/sub-158/L2_task-sharedreward_model-1_type-ppi_seed-VS_sm-4.gfeat/cope4.feat/stats/cope1.nii.gz,158,3
231,/gpfs/scratch/tug87422/smithlab-shared/sharedreward-aging/derivatives/fsl/sub-159/L2_task-sharedreward_model-1_type-ppi_seed-VS_sm-4.gfeat/cope4.feat/stats/cope1.nii.gz,159,3


In [210]:
# Merge the metrics df with the L3inputs (fsl) df and drop unecessary columns
metric_run_merge_df = pd.merge(fsl_df, parsed_metrics_df, on='sub')
metric_run_merge_df = pd.merge(fsl_df, parsed_metrics_df, on='sub')


# Create the new columns based on 'Run'
metric_run_merge_df['fdmean'] = metric_run_merge_df.apply(
    lambda row: row['fd_mean_1'] if row['run'] == 1 else (row['fd_mean_2'] if row['run'] == 2 else row['avg_fdmean']),
    axis=1
)


# Add dummy coding for study ID (SRNDNA=0 and RF1=1)
metric_run_merge_df['study'] = metric_run_merge_df['sub'].apply(lambda x: 0 if len(str(x)) == 3 else 1)

# Add columns of ones
metric_run_merge_df['ones'] = metric_run_merge_df['sub'].apply(lambda x: '1')



In [211]:
## Lets change the filepaths so that they work with the L3 templates. For this, we need to make sure it can switch between act and ppi, as well as L1 and L2 inputs

# Replace analyses type
metric_run_merge_df['path'] = metric_run_merge_df['path'].str.replace('type-ppi_seed-VS', 'type-REPLACEME')

# Replace paths to copenum for L2
metric_run_merge_df['path'] = metric_run_merge_df['path'].str.replace('sm-4.gfeat/cope4.feat/stats/cope1.nii.gz', 'sm-4.gfeat/copeCOPENUM.feat/stats/cope1.nii.gz')

# Replace paths to copenum for L1
metric_run_merge_df['path'] = metric_run_merge_df['path'].str.replace('L1_task-sharedreward_model-1_type-REPLACEME_run-1_sm-4.feat/stats/cope1.nii.gz', 'L1_task-sharedreward_model-1_type-REPLACEME_run-1_sm-4.feat/stats/copeCOPENUM.nii.gz')


display(metric_run_merge_df)

,path,sub,run,tsnr_1,tsnr_2,fd_mean_1,fd_mean_2,Status_1,Status_2,avg_fdmean,fdmean,study,ones
0,/gpfs/scratch/tug87422/smithlab-shared/sharedreward-aging/derivatives/fsl/sub-10317/L2_task-sharedreward_model-1_type-REPLACEME_sm-4.gfeat/copeCOPENUM.feat/stats/cope1.nii.gz,10317,3,17.756101,18.523997,0.187426,0.200535,Include,Include,0.193980,0.193980,1,1
1,/gpfs/scratch/tug87422/smithlab-shared/sharedreward-aging/derivatives/fsl/sub-10369/L2_task-sharedreward_model-1_type-REPLACEME_sm-4.gfeat/copeCOPENUM.feat/stats/cope1.nii.gz,10369,3,18.892139,19.339240,0.112487,0.141168,Include,Include,0.126827,0.126827,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,/gpfs/scratch/tug87422/smithlab-shared/sharedreward-aging/derivatives/fsl/sub-158/L2_task-sharedreward_model-1_type-REPLACEME_sm-4.gfeat/copeCOPENUM.feat/stats/cope1.nii.gz,158,3,52.446490,49.590779,0.129296,0.217482,Include,Include,0.173389,0.173389,0,1
231,/gpfs/scratch/tug87422/smithlab-shared/sharedreward-aging/derivatives/fsl/sub-159/L2_task-sharedreward_model-1_type-REPLACEME_sm-4.gfeat/copeCOPENUM.feat/stats/cope1.nii.gz,159,3,81.834075,77.469568,0.098513,0.108801,Include,Include,0.103657,0.103657,0,1


In [212]:
# Define the base log directory
log_dir = os.path.join(os.getcwd(), '..', 'stimuli', 'logs-reformatted')
log_dir = os.path.abspath(log_dir)

# Define the pattern for CSV files (sub*_SR-Ratings-*.csv in any subfolder)
file_pattern = os.path.join(log_dir, '*', 'sub*_SR-Ratings-*.csv')

# Use glob to get all file paths that match the pattern
file_paths = glob.glob(file_pattern)

# Initialize an empty list to store dataframes
log_df_list = []

# Loop through all matching files and read them into pandas DataFrames
for file_path in file_paths:
    try:
        # Read the CSV file
        log_df = pd.read_csv(file_path)
        
        # Extract the subject number from the file name using regular expressions
        match = re.search(r'sub(\d+)_', os.path.basename(file_path))
        if match:
            subject_id = match.group(1)  # Extract the digits (subject ID)
        else:
            subject_id = None  # If no match is found, set to None or handle as needed
        
        # Add the "sub" column to the DataFrame
        log_df['sub'] = subject_id
        
        # Append the dataframe to the list
        log_df_list.append(log_df)
    except Exception as e:
        print(f"Error reading {file_path}: {e}")

# Concatenate all DataFrames into one
if log_df_list:
    concatenated_log_df = pd.concat(log_df_list, ignore_index=True)
    print(f"Concatenated DataFrame shape: {concatenated_log_df.shape}")
    
    # Optionally, save the concatenated DataFrame to a new CSV
    concatenated_log_df.to_csv(os.path.join(log_dir, 'concatenated_data.csv'), index=False)
else:
    print("No files were found or could be read.")

# Print the first 100 rows of the concatenated DataFrame
# print(concatenated_log_df.head(1500))

# List of rows to drop (you can combine single indices and slices)
rows_to_drop = [*range(6, 12), # 104 
                *range(108, 115),  # 11348 
                *range(275, 282), # 
                *range(283, 290),  # 283:289, inclusive range
                *range(626, 633),  # 626:632, inclusive range
                *range(1011, 1018),  # 1011:1017, inclusive range
                *range(1036, 1043)]  # 1036:1042, inclusive range

# Drop the rows from the DataFrame using the drop() method
concatenated_log_df_clean = concatenated_log_df.drop(rows_to_drop, axis=0)

# Reset the index after dropping rows
concatenated_log_df_clean.reset_index(drop=True, inplace=True)

# Print the first few rows to confirm the changes
print(concatenated_log_df_clean)


Concatenated DataFrame shape: (1469, 7)
     trial partner trait  ran order response    sub
0        1       3     0  1.0   0.0      5.0  11376
1        2       2     0  1.0   1.0      5.0  11376
...    ...     ...   ...  ...   ...      ...    ...
1419     5       2     1  1.0   4.0      0.0  10677
1420     6       1     1  1.0   5.0      0.0  10677

[1421 rows x 7 columns]


In [213]:
# This cell is used to clean the logs files so that each sub only has one row
# Need to add code to drop index column

# Pivot the dataframe to reshape it so that each subject has a single row
log_df = concatenated_log_df_clean.pivot_table(index='sub', columns=['partner', 'trait'], values='response')

# Flatten the MultiIndex columns
log_df.columns = [f'partner_{col[0]}_trait_{col[1]}' for col in log_df.columns]



,partner_1_trait_0,partner_1_trait_1,partner_2_trait_0,partner_2_trait_1,partner_3_trait_0,partner_3_trait_1,partner_1_trait_0,partner_1_trait_1,partner_2_trait_0,partner_2_trait_1,partner_3_trait_0,partner_3_trait_1
sub,,,,,,,,,,,,
10296,1.0,1.0,0.0,-2.0,0.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN
103,1.0,0.0,1.0,0.0,3.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
158,1.0,0.0,2.0,0.0,4.0,-3.0,NaN,NaN,NaN,NaN,NaN,NaN
159,3.0,-3.0,3.0,-3.0,4.0,-5.0,NaN,NaN,NaN,NaN,NaN,NaN


In [218]:

# Assuming 'log_df' is your original DataFrame
# Move 'sub' into the header by resetting the index
df_cleaned = log_df.reset_index()

# Before removing rows, capture the 'sub' values that will be dropped
subs_to_drop = df_cleaned[df_cleaned.drop('sub', axis=1).nunique(axis=1) == 1]['sub']

# Drop rows where all values (except 'sub') are the same
df_cleaned = df_cleaned.loc[~(df_cleaned.drop('sub', axis=1).nunique(axis=1) == 1)]

# Print the cleaned DataFrame
display(df_cleaned)

# Output the 'sub' values that were dropped
print(f"Subs that were dropped: {subs_to_drop.tolist()}")




,sub,partner_1_trait_0,partner_1_trait_1,partner_2_trait_0,partner_2_trait_1,partner_3_trait_0,partner_3_trait_1,partner_1_trait_0,partner_1_trait_1,partner_2_trait_0,partner_2_trait_1,partner_3_trait_0,partner_3_trait_1
0,10296,1.0,1.0,0.0,-2.0,0.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,103,1.0,0.0,1.0,0.0,3.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,158,1.0,0.0,2.0,0.0,4.0,-3.0,NaN,NaN,NaN,NaN,NaN,NaN
236,159,3.0,-3.0,3.0,-3.0,4.0,-5.0,NaN,NaN,NaN,NaN,NaN,NaN


Subs that were dropped: ['10860', '10913', '10938', '10940', '11071', '11085', '126', '140']


In [220]:
import pandas as pd

# Assuming 'log_df' is your original DataFrame
# Move 'sub' into the header by resetting the index
df_cleaned = log_df.reset_index()

# Before removing rows, capture the 'sub' values that will be dropped due to identical ratings
subs_to_drop_same = df_cleaned[df_cleaned.drop('sub', axis=1).nunique(axis=1) == 1]['sub']

# Drop rows where all values (except 'sub') are the same
df_cleaned = df_cleaned.loc[~(df_cleaned.drop('sub', axis=1).nunique(axis=1) == 1)]

# Now, drop subs where trait_1 sum is greater than trait_0 sum
# First, sum the trait_0 and trait_1 columns for each 'sub'
trait_0_columns = [col for col in df_cleaned.columns if 'trait_0' in col]
trait_1_columns = [col for col in df_cleaned.columns if 'trait_1' in col]

# Calculate the sum for trait_0 and trait_1 for each sub
df_cleaned['trait_0_sum'] = df_cleaned[trait_0_columns].sum(axis=1)
df_cleaned['trait_1_sum'] = df_cleaned[trait_1_columns].sum(axis=1)

# Capture the 'sub' values to drop where trait_1_sum > trait_0_sum
subs_to_drop_sum = df_cleaned[df_cleaned['trait_1_sum'] > df_cleaned['trait_0_sum']]['sub']

# Drop the rows where trait_1 sum is greater than trait_0 sum
df_cleaned = df_cleaned[df_cleaned['trait_1_sum'] <= df_cleaned['trait_0_sum']]

# Print the cleaned DataFrame
display(df_cleaned)

# Output the 'sub' values that were dropped
print(f"Subs that were dropped due to identical ratings: {subs_to_drop_same.tolist()}")
print(f"Subs that were dropped due to trait_1 sum > trait_0 sum: {subs_to_drop_sum.tolist()}")



,sub,partner_1_trait_0,partner_1_trait_1,partner_2_trait_0,partner_2_trait_1,partner_3_trait_0,partner_3_trait_1,partner_1_trait_0,partner_1_trait_1,partner_2_trait_0,partner_2_trait_1,partner_3_trait_0,partner_3_trait_1,trait_0_sum,trait_1_sum
0,10296,1.0,1.0,0.0,-2.0,0.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,-4.0
1,103,1.0,0.0,1.0,0.0,3.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,10.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,158,1.0,0.0,2.0,0.0,4.0,-3.0,NaN,NaN,NaN,NaN,NaN,NaN,14.0,-6.0
236,159,3.0,-3.0,3.0,-3.0,4.0,-5.0,NaN,NaN,NaN,NaN,NaN,NaN,20.0,-22.0


Subs that were dropped due to identical ratings: ['10860', '10913', '10938', '10940', '11071', '11085', '126', '140']
Subs that were dropped due to trait_1 sum > trait_0 sum: ['11125', '11134', '11203', '11209', '127']


In [ ]:

# Set 'sub' and 'age' as indices and calculate z-scores only for non-index columns
#z_scores_df = log_df.set_index(['sub', 'age']).apply(zscore, axis=1).reset_index()

# Rename columns to indicate z-scored values
#z_scores_df.columns = ['sub', 'age'] + [f"{col}" for col in log_df.columns[2:]]

#z_scores_df = z_scores_df.iloc[:, :-1]

# Display the final dataframe
#display(z_scores_df.head())